In [ ]:
from IPython.display import HTML
HTML(filename='/Users/hubatsl/Desktop/SPT/Us/SPT/Python/hide_code.html')

# Test for databank system

In [ ]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys
import tifffile
import time
# Add path to python modules
sys.path.append(os.getcwd()[0:-7]+'src/') 
from MovieTracks import (DiffusionFitter, OffRateFitter, ParticleFinder)
# Plotting becomes part of the notebook
%matplotlib inline

**Run thresholding on movie movieNo, read and write threshold from excel file**

In [ ]:
xls_file = '/Users/hubatsl/Desktop/ExSPT.xlsx'
xls_db = pd.read_excel(xls_file)
movieNo = 0
if np.isnan(xls_db['Threshold'][movieNo]):
    xls_db.loc[movieNo, 'Threshold']=300
thresh = int(xls_db['Threshold'][movieNo])
if np.isnan(xls_db['StartFrame'][movieNo]):
    xls_db.loc[movieNo, 'StartFrame'] = 0
startFrame = int(xls_db.loc[movieNo, 'StartFrame'])

def adj_thresh(calibFrame, thresh, startFrame):
    calibFrame = int(calibFrame)
    thresh = int(thresh)
    startFrame = int(startFrame)
    o = OffRateFitter(xls_db['Folder'][movieNo], thresh, pixelSize=0.120, startFrame=startFrame)
    print(xls_db['Folder'][movieNo])
    o.plot_calibration(calibrationFrame=calibFrame)
    xls_db.loc[movieNo, 'StartFrame'] = startFrame
    xls_db.loc[movieNo, 'Threshold'] = thresh
    xls_db.to_excel(xls_file)

interact(adj_thresh, thresh=str(thresh), calibFrame='200', startFrame=str(startFrame),
         __manual=True)

**Read time stamps for each movie, write to excel file**

In [ ]:
xls_db['IntervalReal'] = 'nan'
xls_db['ExposureReal'] = 'nan'
xls_db['Length'] = 'nan'
for i, path in enumerate(xls_db['Folder']):
    p = ParticleFinder(path)
    xls_db.loc[i, 'IntervalReal'] = p.timestep
    xls_db.loc[i, 'ExposureReal'] = p.exposure
    xls_db.loc[i, 'Length'] = len(p.frames)
xls_db.to_excel(xls_file)

**Run single particle tracking on all movies specified in SPT.xlsx**

Minimum track length is set either to (8x0.033)/interval frames (so the minimum time one particle is tracked is 2.64 s) or to 20 frames, if (20 x interval) > 2.64.

In [ ]:
for i, path in enumerate(xls_db['Folder']):
    interval = xls_db['IntervalReal'][i]
    if 2.64/interval<20: mTL = 20
    else: mTL = 2.64/interval
    p = DiffusionFitter(path, xls_db['Threshold'][i],
                        minTrackLength=mTL)
    p.analyze()
    p.append_output_to_csv('/Users/hubatsl/Desktop/ExSPT_to_delete.csv', p.gData())

**Run single particle tracking for ith movie in file**

In [ ]:
i = 1
path = xls_db['Folder'][i]
interval = xls_db['IntervalReal'][i]
if 2.64/interval<20: mTL = 20
else: mTL = 2.64/interval
mTL = 80
p = DiffusionFitter(path, xls_db['Threshold'][i],
                        minTrackLength=mTL, startFrame=1300)
p.plot_calibration()
p.analyze()
p.showFigs=True
p.plot_diffusion_vs_alpha()
p.D_restricted